# Preprocessing the MTA Hourly Dataset from Kaggle

## Initial Steps

### Imports and Loading the CSV:

We start by importing all the libraries we'll need. I'm also going to create a function to load the CSV in.

In [27]:
#All datascience imports
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [ ]:
#Function to load CSV
def load_csv(file_name):
    """
    Load a CSV file from the 'CSVs' folder into a pandas DataFrame.
    """
    try:
        file_path = os.path.join("..", "CSVs", file_name)
        df = pd.read_csv(file_path)
        print(f"CSV file '{file_path}' loaded successfully.")
        return df
    except Exception as e:
        print(f"Error loading CSV file: {e}")
        return None

In [29]:
#Load the CSV file renamed to 'data.csv' from the current directory
csv_file_path = 'data.csv'
raw_data = load_csv(csv_file_path)

C:\Users\Esh\AppData\Local\Temp\ipykernel_29000\1242278213.py:13: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


CSV file 'data.csv' loaded successfully.


### Taking a Look at our Data:

We've got a large dataset, and we're already getting a DtypeWarning from columns having mixed types. This next step involves us peeking into the dataset, and seeing what we've got.

In [30]:
#Look into the data now by creating a function to display all the types of initial statistics such as head, tail, info, and describe and more.
def display_data_info(df):
    """
    Display various statistics and information about the DataFrame.
    
    Parameters:
    df (pd.DataFrame): The DataFrame to analyze.
    """
    print("DataFrame Head:")
    print(df.head())
    
    print("\nDataFrame Tail:")
    print(df.tail())
    
    print("\nDataFrame Info:")
    print(df.info())
    
    print("\nDataFrame Description:")
    print(df.describe())
    
    print("\nDataFrame Columns:")
    print(df.columns)

#Taking a Look at our Data:
display_data_info(raw_data)

DataFrame Head:
        transit_timestamp transit_mode station_complex_id  \
0  09/23/2023 05:00:00 AM       subway                185   
1  09/23/2023 05:00:00 PM       subway                131   
2  05/13/2023 09:00:00 AM       subway                173   
3  05/13/2023 10:00:00 PM       subway                135   
4  05/13/2023 12:00:00 PM       subway                349   

               station_complex   borough payment_method  \
0               Liberty Av (C)  Brooklyn           omny   
1  Bushwick Av-Aberdeen St (L)  Brooklyn      metrocard   
2                High St (A,C)  Brooklyn           omny   
3               Livonia Av (L)  Brooklyn      metrocard   
4                Junius St (3)  Brooklyn      metrocard   

            fare_class_category  ridership  transfers   latitude  longitude  \
0   OMNY - Seniors & Disability          1          0  40.674540 -73.896545   
1  Metrocard - Unlimited 30-Day          4          0  40.682830 -73.905250   
2              OMNY - Ful

### Let's Make the Data Easy to Read

We're on a zero index right now, and the dataset consists of mainly all strings. It's time to start formatting the data to make it easier to clean.

In [31]:
# Make a working copy so the original stays untouched
modded_data = raw_data.copy()

# Normalize headers to snake_case
modded_data.columns = (
    modded_data.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
      .str.replace(r"[^\w]+", "_", regex=True)
      .str.strip("_")
)

# Rename to proper name we can use later
rename_map = {
    "transit_timestamp": "timestamp",
    "station_complex_id": "station_id",
    "station_complex": "station",
    "payment_method": "payment_type"
}
modded_data = modded_data.rename(columns=rename_map)

# We don't need the georeference column for our analysis because it's essentially a duplicate of latitude and longitude
if "georeference" in modded_data.columns:
    modded_data = modded_data.drop(columns=["georeference"])

# Let's take a look at the first few rows of the modified data
print("Modified Data Head:")
modded_data.head()

Modified Data Head:


,timestamp,transit_mode,station_id,station,borough,payment_type,fare_class_category,ridership,transfers,latitude,longitude
0,09/23/2023 05:00:00 AM,subway,185,Liberty Av (C),Brooklyn,omny,OMNY - Seniors & Disability,1,0,40.674540,-73.896545
1,09/23/2023 05:00:00 PM,subway,131,Bushwick Av-Aberdeen St (L),Brooklyn,metrocard,Metrocard - Unlimited 30-Day,4,0,40.682830,-73.905250
2,05/13/2023 09:00:00 AM,subway,173,"High St (A,C)",Brooklyn,omny,OMNY - Full Fare,82,0,40.699337,-73.990530
3,05/13/2023 10:00:00 PM,subway,135,Livonia Av (L),Brooklyn,metrocard,Metrocard - Unlimited 30-Day,5,0,40.664040,-73.900570
4,05/13/2023 12:00:00 PM,subway,349,Junius St (3),Brooklyn,metrocard,Metrocard - Full Fare,7,4,40.663513,-73.902450


The data already looks clean, but the formatting is still not the best it can be. We have a timestamp in a string, but we can convert that to date time format as well for easier parsing.

In [32]:
#Convert timestamp to datetime format
modded_data['timestamp'] = pd.to_datetime(modded_data['timestamp'], errors='coerce')

In [33]:
print("Modified Data Head:")
modded_data.head()

Modified Data Head:


,timestamp,transit_mode,station_id,station,borough,payment_type,fare_class_category,ridership,transfers,latitude,longitude
0,2023-09-23 05:00:00,subway,185,Liberty Av (C),Brooklyn,omny,OMNY - Seniors & Disability,1,0,40.674540,-73.896545
1,2023-09-23 17:00:00,subway,131,Bushwick Av-Aberdeen St (L),Brooklyn,metrocard,Metrocard - Unlimited 30-Day,4,0,40.682830,-73.905250
2,2023-05-13 09:00:00,subway,173,"High St (A,C)",Brooklyn,omny,OMNY - Full Fare,82,0,40.699337,-73.990530
3,2023-05-13 22:00:00,subway,135,Livonia Av (L),Brooklyn,metrocard,Metrocard - Unlimited 30-Day,5,0,40.664040,-73.900570
4,2023-05-13 12:00:00,subway,349,Junius St (3),Brooklyn,metrocard,Metrocard - Full Fare,7,4,40.663513,-73.902450


### Fixing other Column dtypes

timestamp looks good, but lets check to make sure all other columns are what they're supposed to be. We have a lot of numeric columns, so lets ensure they parse numeric.

In [34]:
#Get dtypes of all columns
print("Column Data Types:")
print(modded_data.dtypes)

Column Data Types:
timestamp              datetime64[ns]
transit_mode                   object
station_id                     object
station                        object
borough                        object
payment_type                   object
fare_class_category            object
ridership                       int64
transfers                       int64
latitude                      float64
longitude                     float64
dtype: object


In [35]:
for col in ["ridership", "transfers", "latitude", "longitude"]:
    if col in modded_data.columns:
        modded_data[col] = pd.to_numeric(modded_data[col], errors="coerce")

cat_cols = ["transit_mode", "station_id", "station", "borough", "payment_type", "fare_class_category"]
for col in cat_cols:
    if col in modded_data.columns:
        modded_data[col] = modded_data[col].astype("category")

In [36]:
#Get dtypes of all columns
print("Column Data Types:")
print(modded_data.dtypes)

Column Data Types:
timestamp              datetime64[ns]
transit_mode                 category
station_id                   category
station                      category
borough                      category
payment_type                 category
fare_class_category          category
ridership                       int64
transfers                       int64
latitude                      float64
longitude                     float64
dtype: object


### Cleaning up Duplicates (if any)

We've set up our categorical and numerical columns. This next step is just to ensure we don't have any duplicates.

I initially attempted to drop duplicates as-is, but found that there are 36M potential duplicate rows in our key columns. So to better understand this, we need to go into our duplicates and see what we're looking at.

In [37]:
# Remove rows that are byte-for-byte identical (safe)
before = len(modded_data)
modded_data = modded_data.drop_duplicates()
print(f"Exact duplicate rows removed: {before - len(modded_data):,}")

Exact duplicate rows removed: 0


With no exact duplicate rows found, we need to aggregate across fare classes. This is where I currently suspect the "duplicate data" is located.

In [ ]:
agg = (
    modded_data
      .groupby(["station_id","station","borough","timestamp","payment_type"], as_index=False, observed=True)["ridership"]
      .sum()
)
print("Rows after summing fare classes:", f"{len(agg):,}")

Rows after summing fare classes: 14,708,658


This concurs with the removal of "duplicate data" I had initially.

In [39]:
print("Original sum:", modded_data["ridership"].sum())
print("Aggregated sum:", agg["ridership"].sum())

Original sum: 2291540464
Aggregated sum: 2291540464


No ridership lost! It's time to build our wide table.

## Wide Formatting

### Table Manipulation

So we've got our modded_Data for a long table format, but for the purpose of our capstone, we need a wide table for forecasting. We need to approach with caution. My initial attempt resulted in a MemoryError. 
**(Can't forget we have 14M rows!)**

In [40]:
agg_data = agg.copy() #I prefer the naming convention of agg_data for clarity, plus we have a copy of the original data in case we need it later

totals = (
    agg_data.groupby(["station_id","station","borough","timestamp"], as_index=False, observed=True)["ridership"]
        .sum()
        .rename(columns={"ridership":"ridership_total"})
) #Ridership totals across all fare classes

print("Total ridership data shape:", totals.shape)

Total ridership data shape: (7614788, 5)


In [42]:
# Work from the fare-class-aggregated frame
data = agg_data.copy()


KEYS = ["station_id","station","borough","timestamp"]

# Total ridership per station-hour (target for modeling)
totals = (
    data.groupby(KEYS, observed=True, sort=False)["ridership"]
        .sum()
        .rename("ridership_total")
        .reset_index()
)
print("totals rows:", len(totals))


totals rows: 7614788


Let's gradually add in our per payment columns.

In [43]:
ptypes = list(data["payment_type"].cat.categories)

wide = totals

for ptype in ptypes:
    wide = (
        wide.merge(
            data[data["payment_type"] == ptype][KEYS + ["ridership"]],
            on=KEYS,
            how="left",
            suffixes=("", f"_{ptype}")
        )
    )

print("wide rows:", len(wide), "| columns:", wide.shape[1])
wide.head()

wide rows: 7614788 | columns: 7


,station_id,station,borough,timestamp,ridership_total,ridership,ridership_omny
0,1,"Astoria-Ditmars Blvd (N,W)",Queens,2022-05-18 00:00:00,42,27.0,15.0
1,1,"Astoria-Ditmars Blvd (N,W)",Queens,2022-05-18 01:00:00,11,9.0,2.0
2,1,"Astoria-Ditmars Blvd (N,W)",Queens,2022-05-18 02:00:00,5,4.0,1.0
3,1,"Astoria-Ditmars Blvd (N,W)",Queens,2022-05-18 03:00:00,20,15.0,5.0
4,1,"Astoria-Ditmars Blvd (N,W)",Queens,2022-05-18 04:00:00,57,41.0,16.0


Looking back at it, the ridership_total was the only necessary column needed. As useful as it is for regular v.s. omny, our model is strictly focusing on totals. However, this could be adopted for a different model, so it's not bad to know what the ridership_total consists of.

### What time is it now?

So now we need to decide if we want to forecast our hourly ridership, or if we want just our daily totals. We can potentially resample the data for daily forecasting, but it'd be a shame to waste data on hourly trends, so let's stick with hourly forecasting.

Let's break down our timestamps and see what we have.

In [ ]:
timed_data = wide.copy()

timed_data["hour"] = timed_data["timestamp"].dt.hour
timed_data["day"] = timed_data["timestamp"].dt.day
timed_data["day_of_week"] = timed_data["timestamp"].dt.dayofweek
timed_data["month"] = timed_data["timestamp"].dt.month
timed_data["year"] = timed_data["timestamp"].dt.year
timed_data["is_weekend"] = timed_data["day_of_week"].isin([5, 6]).astype(int) #an interesting indicator, yes, but it can be useful for forecasting ridership trends on weekends

#We just split up our timestamp into better columns for analysis
print("Timed Data Head:")
timed_data.head()

Timed Data Head:


,station_id,station,borough,timestamp,ridership_total,ridership,ridership_omny,hour,day,day_of_week,month,year,is_weekend
0,1,"Astoria-Ditmars Blvd (N,W)",Queens,2022-05-18 00:00:00,42,27.0,15.0,0,18,2,5,2022,0
1,1,"Astoria-Ditmars Blvd (N,W)",Queens,2022-05-18 01:00:00,11,9.0,2.0,1,18,2,5,2022,0
2,1,"Astoria-Ditmars Blvd (N,W)",Queens,2022-05-18 02:00:00,5,4.0,1.0,2,18,2,5,2022,0
3,1,"Astoria-Ditmars Blvd (N,W)",Queens,2022-05-18 03:00:00,20,15.0,5.0,3,18,2,5,2022,0
4,1,"Astoria-Ditmars Blvd (N,W)",Queens,2022-05-18 04:00:00,57,41.0,16.0,4,18,2,5,2022,0


In [51]:
#One thing I wanted to test out is the holidays indicator as well.
#We can use the holidays package to get a list of holidays in the US, and then create a boolean column indicating if the date is a holiday or not.
import holidays
us_holidays = holidays.US(years=timed_data["year"].unique().tolist())
timed_data["is_holiday"] = timed_data["timestamp"].dt.date.isin(us_holidays).astype(int)

print("Timed Data with Holidays Head:", timed_data.head())


#check if the is_holiday column is working correctly
print("Is Holiday Column Unique Values:", timed_data["is_holiday"].unique())

Timed Data with Holidays Head:   station_id                     station borough           timestamp  \
0          1  Astoria-Ditmars Blvd (N,W)  Queens 2022-05-18 00:00:00   
1          1  Astoria-Ditmars Blvd (N,W)  Queens 2022-05-18 01:00:00   
2          1  Astoria-Ditmars Blvd (N,W)  Queens 2022-05-18 02:00:00   
3          1  Astoria-Ditmars Blvd (N,W)  Queens 2022-05-18 03:00:00   
4          1  Astoria-Ditmars Blvd (N,W)  Queens 2022-05-18 04:00:00   

   ridership_total  ridership  ridership_omny  hour  day  day_of_week  month  \
0               42       27.0            15.0     0   18            2      5   
1               11        9.0             2.0     1   18            2      5   
2                5        4.0             1.0     2   18            2      5   
3               20       15.0             5.0     3   18            2      5   
4               57       41.0            16.0     4   18            2      5   

   year  is_weekend  is_holiday  
0  2022           0  

## Final Analysis

### What We Did

In this stage of preprocessing, we worked through several key transformations to make the raw subway ridership data usable for analysis and forecasting. The dataset originally contained over fifty million entries and that is very suffocating, with each record tied to a particular fare class and payment method. To reduce redundancy and create a cleaner structure, we first removed duplicates and aggregated across fare classes, which brought the dataset down to about 14.7 million rows while still preserving the full ridership totals. This step ensured that the information we kept was unique

After fixing the time dimension, we reshaped the data into a wide format by pivoting payment types into their own columns. This means that each row now corresponds to a station and an hour, with ridership broken down into totals, MetroCard entries, and OMNY entries for our forecasting models.

### Why We Did

The decisions we made in this stage all had one goal: to simplify the dataset into a  model-ready format without losing any potentially important detail, and giving us more inferences from what we have. The cleaned and reshaped data now reflects the real points of analysis.

### What's Next

The data above should be ready for our model to train on. So this concludes this part of the capstone with me exporting our final table as another csv to use. 

In [ ]:
#Create the function to save the CSVs we've made.
def save_csv(df, file_name):
    """
    Save a DataFrame to a CSV file inside the 'CSVs' folder.
    """
    try:
        file_path = os.path.join("..", "CSVs", file_name)  # go up one dir from Notebooks/
        df.to_csv(file_path, index=False)
        print(f"DataFrame saved to '{file_path}' successfully.")
    except Exception as e:
        print(f"Error saving DataFrame to CSV: {e}")

# Save the processed DataFrame to a CSV file
save_csv(timed_data, 'processed_data.csv')
# Save the wide DataFrame to a CSV file
save_csv(wide, 'wide_data.csv')

DataFrame saved to 'processed_data.csv' successfully.
DataFrame saved to 'wide_data.csv' successfully.
